## 1 Importing and filtering data:

In [1]:
import pandas as pd
import numpy as np 

import_file_path = r"C:\Users\olive\OneDrive\Documents\University Work\Y3S2\JIV_RA_task\usa_wk_apr_2013.xlsx"
export_file_path = r"C:\Users\olive\OneDrive\Documents\University Work\Y3S2\JIV_RA_task\KLEMS_indeces_2013_samecode.xlsx"

KLEMS_data = (
    pd.read_excel(import_file_path, sheet_name="DATA")
    .query("code == 'AtB'")
    .melt(id_vars=["Variable", "desc", "code"], var_name ="Year", value_name="Value")
    .pivot(index=("Year", "code"), columns="Variable", values="Value")
    .loc[:, ["GO", "GO_QI", "II", "II_QI", "LAB", "LAB_QI", "CAP", "CAP_QI"]])
KLEMS_data = KLEMS_data.rename(columns={
    "GO": "GO",
    "CAP": "CAP",
    "LAB": "LAB",
    "II": "II",
    "GO_QI": "GO_real",
    "CAP_QI": "CAP_real",
    "LAB_QI": "LAB_real",
    "II_QI": "II_real"}).copy()

## 2 Computing productivity growth rates

### 2.1 'Nominal value added' and 'nominal value added share of output'

#### 2.1.1 Nominal value added

Formula 

$$
P_{V A}(t) Q_{V A}(t) = P_Y (t) Q_Y (t) - P_{II} (t) Q_{II} (t)
$$

In [2]:
KLEMS_data.loc[:, "nominal_VA"] = (KLEMS_data["GO"] - KLEMS_data["II"])

#### 2.1.2 Nominal value added share of output

Formula: 

$$
\nu_{V A}(t) = \frac{P_{V A}(t) Q_{V A}(t)}{P_Y (t) Q_Y (t)}
$$

In [3]:
KLEMS_data.loc[:, "VA_output_share"] = (KLEMS_data["nominal_VA"]/KLEMS_data["GO"])

### 2.2 Value added quantity growth rate

Formula

$$
\Delta \ln Q_{VA}(t) = \frac{\Delta \ln Q_Y (t) - \bar{\nu}_{II} (t) \Delta \ln Q_{II} (t)}{\bar{\nu}_{V A} (t)}
$$

where:

$$
\Delta \ln X(t) = \ln X(t) - \ln X(t - 1)
$$

and:

$$
\bar{\nu}_X(t) = 0.5 \times \left( \frac{P_X(t) Q_X(t)}{P_Y (t) Q_Y (t)} + \frac{P_X(t - 1) Q_X(t - 1)}{P_Y (t - 1) Q_Y (t - 1)} \right)
$$

#### 2.2.1 Log differences

$$
\Delta \ln X(t) = \ln X(t) - \ln X(t - 1)
$$

where X is either real GO or real II

In [4]:
KLEMS_data.loc[:, "ln_GO_real"] = np.log(KLEMS_data["GO_real"])
KLEMS_data.loc[:, "ln_II_real"] = np.log(KLEMS_data["II_real"])

KLEMS_data.loc[:, "delta_ln_GO_real"] = (KLEMS_data.groupby("code")["ln_GO_real"].transform(lambda x: x - x.shift(1)))
KLEMS_data.loc[:, "delta_ln_II_real"] = (KLEMS_data.groupby("code")["ln_II_real"].transform(lambda x: x - x.shift(1)))

#### 2.2.2 Tornqvist Output Share

$$
\bar{\nu}_X(t) = 0.5 \times \left( \frac{P_X(t) Q_X(t)}{P_Y (t) Q_Y (t)} + \frac{P_X(t - 1) Q_X(t - 1)}{P_Y (t - 1) Q_Y (t - 1)} \right)
$$

where X is either nominal VA or nominal II

In [5]:
KLEMS_data.loc[:, "VA_over_GO"] = KLEMS_data["nominal_VA"] / KLEMS_data["GO"]
KLEMS_data.loc[:, "II_over_GO"] = KLEMS_data["II"] / KLEMS_data["GO"]

KLEMS_data.loc[:, "VA_over_GO_lag"] = KLEMS_data.groupby("code")["VA_over_GO"].transform(lambda x: x.shift(1))
KLEMS_data.loc[:, "II_over_GO_lag"] = KLEMS_data.groupby("code")["II_over_GO"].transform(lambda x: x.shift(1))

KLEMS_data.loc[:, "VA_tornqvist_output_share"] = 0.5 * (KLEMS_data["VA_over_GO"] + KLEMS_data["VA_over_GO_lag"])
KLEMS_data.loc[:, "II_tornqvist_output_share"] = 0.5 * (KLEMS_data["II_over_GO"] + KLEMS_data["II_over_GO_lag"])

#### 2.2.3 Value added quantity growth rate

$$
\Delta \ln Q_{VA}(t) = \frac{\Delta \ln Q_Y (t) - \bar{\nu}_{II} (t) \Delta \ln Q_{II} (t)}{\bar{\nu}_{V A} (t)}
$$

In [6]:
KLEMS_data.loc[:, "delta_ln_Q_va"] = ((KLEMS_data["delta_ln_GO_real"] - (KLEMS_data["II_tornqvist_output_share"]*KLEMS_data["delta_ln_II_real"]))/KLEMS_data["VA_tornqvist_output_share"])

### 2.3 Labour productivity growth rate

Formula:

$$
\Delta \ln LP(t) = \Delta \ln Q_{VA}(t) - \Delta \ln Q_L(t)
$$

In [7]:
KLEMS_data.loc[:, "LAB_real_lag"] = KLEMS_data.groupby("code")["LAB_real"].transform(lambda x: x.shift(1))
KLEMS_data.loc[:, "ln_LAB_real"] = np.log(KLEMS_data["LAB_real"])
KLEMS_data.loc[:, "ln_LAB_real_lag"] = np.log(KLEMS_data["LAB_real_lag"])

KLEMS_data.loc[:, "delta_ln_LAB_real"] = (KLEMS_data["ln_LAB_real"] - KLEMS_data["ln_LAB_real_lag"])
KLEMS_data.loc[:, "delta_ln_LP"] = (KLEMS_data["delta_ln_Q_va"] - KLEMS_data["delta_ln_LAB_real"])

### 2.4 Total factor productivity growth rate

Formula:

$$
\Delta \ln TFP(t) = \Delta \ln Q_{VA}(t) - \bar{\psi}_L(t) \Delta \ln Q_L(t) - \bar{\psi}_K(t) \Delta \ln Q_K(t)
$$

where:

$$
\bar{\psi}_X(t) = 0.5 \times \left( \frac{P_X(t) Q_X(t)}{P_{VA}(t) Q_{VA}(t)} + \frac{P_X(t-1) Q_X(t-1)}{P_{VA}(t-1) Q_{VA}(t-1)} \right)
$$

where X is either nominal LAB (L) or nominal CAP (K)

#### 2.4.1 Tornqvist VA share

$$
\bar{\psi}_X(t) = 0.5 \times \left( \frac{P_X(t) Q_X(t)}{P_{VA}(t) Q_{VA}(t)} + \frac{P_X(t-1) Q_X(t-1)}{P_{VA}(t-1) Q_{VA}(t-1)} \right)
$$

where X is either nominal LAB (L) or nominal CAP (K)

In [8]:
KLEMS_data.loc[:, "LAB_over_nominal_VA"] = KLEMS_data["LAB"] / KLEMS_data["nominal_VA"]
KLEMS_data.loc[:, "CAP_over_nominal_VA"] = KLEMS_data["CAP"] / KLEMS_data["nominal_VA"]

KLEMS_data.loc[:, "LAB_over_nominal_VA_lag"] = KLEMS_data.groupby("code")["LAB_over_nominal_VA"].transform(lambda x: x.shift(1))
KLEMS_data.loc[:, "CAP_over_nominal_VA_lag"] = KLEMS_data.groupby("code")["CAP_over_nominal_VA"].transform(lambda x: x.shift(1))

KLEMS_data.loc[:, "L_tornqvist_VA_share"] = 0.5 * (KLEMS_data["LAB_over_nominal_VA"] + KLEMS_data["LAB_over_nominal_VA_lag"])
KLEMS_data.loc[:, "K_tornqvist_VA_share"] = 0.5 * (KLEMS_data["CAP_over_nominal_VA"] + KLEMS_data["CAP_over_nominal_VA_lag"])

#### 2.4.2 Real capital; logged and log difference

In [9]:
KLEMS_data.loc[:, "CAP_real_lag"] = KLEMS_data.groupby("code")["CAP_real"].transform(lambda x: x.shift(1))
KLEMS_data.loc[:, "ln_CAP_real"] = np.log(KLEMS_data["CAP_real"])
KLEMS_data.loc[:, "ln_CAP_real_lag"] = np.log(KLEMS_data["CAP_real_lag"])
KLEMS_data.loc[:, "delta_ln_CAP_real"] = (KLEMS_data["ln_CAP_real"] - KLEMS_data["ln_CAP_real_lag"])

#### 2.4.3 TFP growth rate

$$
\Delta \ln TFP(t) = \Delta \ln Q_{VA}(t) - \bar{\psi}_L(t) \Delta \ln Q_L(t) - \bar{\psi}_K(t) \Delta \ln Q_K(t)
$$

In [10]:
KLEMS_data.loc[:, "delta_ln_TFP"] = (KLEMS_data["delta_ln_Q_va"] - (KLEMS_data["L_tornqvist_VA_share"]*KLEMS_data["delta_ln_LAB_real"]) - (KLEMS_data["K_tornqvist_VA_share"]*KLEMS_data["delta_ln_CAP_real"]))

## 3 Turning growth rates into indeces

### 3.1 TFP

In [11]:
KLEMS_data["ln_TFP"] = KLEMS_data.groupby("code")["delta_ln_TFP"].cumsum()
KLEMS_data["index_ln_TFP"] = np.exp(KLEMS_data["ln_TFP"])

### 3.2 LP

In [12]:
KLEMS_data["ln_LP"] = KLEMS_data.groupby("code")["delta_ln_LP"].cumsum()
KLEMS_data["index_ln_LP"] = np.exp(KLEMS_data["ln_LP"])

### 3.3 Set first index value as 1 for each industry

In [13]:
KLEMS_data.loc[KLEMS_data.index.get_level_values('Year') == '_1947', 'index_ln_TFP'] = 1
KLEMS_data.loc[KLEMS_data.index.get_level_values('Year') == '_1947', 'index_ln_LP'] = 1

## 4 Export to Excel

In [14]:
columns_to_export = ["index_ln_TFP", "index_ln_LP"] 
KLEMS_data[columns_to_export].to_excel(export_file_path, sheet_name="Data", index=True)

print(f"File successfully saved to: {export_file_path}")

File successfully saved to: C:\Users\olive\OneDrive\Documents\University Work\Y3S2\JIV_RA_task\KLEMS_indeces_2013_samecode.xlsx
